# UPDATE:

**Later in this kernel I have started off on how to merge all the csv files together in order to create a dataframe collectively for the purpose of modeling!! Feature engineering will be done along the way too!!**

## **All the files at a glance with their relations**

![](https://image.ibb.co/e4BJUb/rrv.jpg)

## Here are some of my insights of the data so far!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

air_reserve = pd.read_csv('../input/air_reserve.csv')
hpg_reserve = pd.read_csv('../input/hpg_reserve.csv')

air_store_info = pd.read_csv('../input/air_store_info.csv')
hpg_store_info = pd.read_csv('../input/hpg_store_info.csv')

df_train = pd.read_csv('../input/air_visit_data.csv').sort_values('visit_date')
df_test = pd.read_csv('../input/sample_submission.csv')

store_id_relation = pd.read_csv('../input/store_id_relation.csv')

date_info = pd.read_csv('../input/date_info.csv')
# Any results you write to the current directory are saved as output.

**Our first job is to understand the relationship between each of these csv files. I have shown the relationship at the start of the kernel.**

## Reservation based files

** First let us look at the reservation based csv files.**

In [ ]:
air_reserve.head(10)

In [ ]:
air_reserve.visit_datetime.dtype

In [ ]:
air_reserve.shape

In [ ]:
air_reserve.air_store_id.nunique()

**From the above few snippets, we see that more than one reservations have been made by the same id on multiple occassions. We need to hold on to the latest reservation made by a particular id.**

** The following snippet produces a pandas series containing reservations made by each unique id, arranged in the descending order of number of reservations made.**

In [ ]:
a = air_reserve['air_store_id'].value_counts()
a.head()

**The id that made the most reservations:**

In [ ]:
air_reserve['air_store_id'].value_counts().idxmax()

**Reservations made by the restaurant with id `air_8093d0b565e9dbdf`:**

In [ ]:
air_reserve.loc[air_reserve['air_store_id'] == 'air_8093d0b565e9dbdf']

## Store info files

**Let us now look into the files containing store information.**

In [ ]:
air_store_info.head(10)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.set_style("whitegrid")
ax = sns.countplot(y = air_store_info['air_genre_name'], data = air_store_info)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15.7, 23.27)
sns.set_style("whitegrid")
ax = sns.countplot(y = air_store_info['air_area_name'], data = air_store_info)

In [ ]:
#ax = sns.regplot(x = "longitude", y = "latitude", data = air_store_info)

sns.lmplot('latitude', # Horizontal axis
           'longitude', # Vertical axis
           data = air_store_info, # Data source
           fit_reg = False, # Don't fix a regression line
           #hue="z", # Set color
           #scatter_kws={"marker": "D", # Set marker style
                       # "s": 100}
          )

In [ ]:
air_store_info.head()

## **Location of restaurants on HPG system **

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
f,ax = plt.subplots(1, 1, figsize=(15,16))
m = Basemap(projection='aeqd', width=2000000, height=2000000, lat_0=37.5, lon_0=138.2)

m.drawcoastlines()
m.fillcontinents(color='gray', lake_color='white', zorder=1)
m.scatter(hpg_store_info.longitude.values, hpg_store_info.latitude.values, marker='o', color='red', zorder=999, latlon=True)
ax.legend()
plt.show()

## **Location of restaurants on AIR system **

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
f,ax = plt.subplots(1, 1, figsize=(15,16))
m = Basemap(projection='aeqd', width=2000000, height=2000000, lat_0=37.5, lon_0=138.2)

m.drawcoastlines()
m.fillcontinents(color='gray', lake_color='white', zorder=1)
m.scatter(air_store_info.longitude.values, air_store_info.latitude.values, marker='o', color='blue', zorder=999, latlon=True)
#m.scatter(air.longitude.values, air.latitude.values, marker='x', color='green', zorder=999, latlon=True)
ax.legend()
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap

# Create a map on which to draw.  We're using a mercator projection, and showing the whole world.
m = Basemap(projection='merc', llcrnrlat=-80, urcrnrlat=80, llcrnrlon=-180, urcrnrlon=180, lat_ts=20, resolution='c')

# Draw coastlines, and the edges of the map.
m.drawcoastlines()
#m.drawmapboundary()

# Convert latitude and longitude to x and y coordinates
x, y = m(list(air_store_info["longitude"].astype(float)), list(air_store_info["latitude"].astype(float)))

# Use matplotlib to draw the points onto the map.
m.scatter(x, y, 1, marker='o', color='red')

# Show the plot.
plt.show()

In [ ]:
''' 
import folium

# Get a basic world map.
air_map = folium.Map(location=[15, 0], zoom_start=1)
# Draw markers on the map.
for name, row in air_store_info.iterrows():
    # For some reason, this one airport causes issues with the map.
    #if row["name"] != "South Pole Station":
        air_map.circle_marker(location=[row["latitude"], row["longitude"]], popup=row["air_store_id"])
# Create and show the map.
air_map.create_map('air.html')
air_map
'''

In [ ]:
len(air_store_info)

In [ ]:
len(hpg_store_info)

In [ ]:
hpg_store_info.hpg_store_id.nunique()

In [ ]:
hpg_store_info.head(10)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.set_style("whitegrid")
ax = sns.countplot(y = hpg_store_info['hpg_genre_name'], data = hpg_store_info)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15.7, 28.27)
sns.set_style("whitegrid")
ax = sns.countplot(y = hpg_store_info['hpg_area_name'], data = hpg_store_info)

## **Store & id relation**

**The file `store_id_relation.csv` how some restaurants in both the reservation sites are the same.**

**First let us check if there are missing values present.**

In [ ]:
store_id_relation.isnull().values.any()

In [ ]:
store_id_relation.head(10)

**How many such restaurants are the same between both these reservation sites?**

In [ ]:
len(store_id_relation)

In [ ]:
store_id_relation.shape

**There are 150 such restaurants that have ids in both the reservation sites.**

## **Date_info file**

**Let us have a look at the `date_info.csv` file. This is important because it is mentioned that certain days were holidays when the restaurants were closed.**

In [ ]:
date_info.isnull().values.any()

**There are no missing values**

In [ ]:
date_info.head(10)

**We can see three columns containing the date, day of the week, and a holiday flag indicating whether that day is a holiday or not.**

** The lenght of this file would give the number of days present here:**

In [ ]:
len(date_info)

In [ ]:
date_info.shape

**Date range:**

In [ ]:
print('The date ranges between {} and {}'.format(min(date_info.calendar_date), max(date_info.calendar_date)))

**Holiday distribution**

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
ax = sns.countplot(x = date_info['holiday_flg'], data = date_info) 

In [ ]:
f, ax = plt.subplots(figsize=(13, 8))
ax = sns.countplot(x = date_info['day_of_week'], hue =date_info['holiday_flg'], data = date_info) 

## **Train file**

In [ ]:
df_train.isnull().values.any()

In [ ]:
df_train.head(10)

**Date ranges in the train file**

In [ ]:
print('The dates in the train file ranges from {} to {}'.format(min(df_train.visit_date), max(df_train.visit_date)))

**As stated the train file contains date from 1st January, 2016 to 22nd April, 2017.**

# How do you frame the training data for modeling?

**We have so many different csv files. I will show how to link all of them and in doing so engineer new features and prepare a dataframe for modeling**

In [ ]:
print('Train data Head : \n', df_train.head())
print('\n')
print('Test data Head : \n', df_test.head())

In [ ]:
print('Train shape : ', df_train.shape)
print('Test shape : ', df_test.shape)

**In the test dataset the `air_store_id` and the `visit_date` have been merged. Let us separate them into two different columns and drop the `id` column.**

In [ ]:
df_test['air_store_id'] = df_test['id'].map(lambda x: '_'.join(x.split('_')[:2]))
df_test['visit_date'] = df_test['id'].map(lambda x: str(x).split('_')[2])

df_test.drop('id', axis=1, inplace=True)

In [ ]:
print('Number of Unique train Ids in Train set: ', df_train.air_store_id.nunique())
print('Number of Unique train Ids in Test set: ', df_test.air_store_id.nunique())

**We will see if the intersection of both the above lists have the same length.**

In [ ]:
train_air_list = df_train.air_store_id.unique().tolist()
test_air_list = df_test.air_store_id.unique().tolist()

len(set(train_air_list) & set(test_air_list))

In [ ]:
if(set(test_air_list) == (set(train_air_list) & set(test_air_list))):
    print('True')
else:
    print('False')

** We have confirmed that the air_id in the test set are the ones taken from the train set!**

In [ ]:
del train_air_list
del test_air_list

** Extracting date features from train and test set**

In [ ]:
df_train['visit_date'] = pd.to_datetime(df_train['visit_date'])
df_train['dow'] = df_train['visit_date'].dt.dayofweek
df_train['year'] = df_train['visit_date'].dt.year
df_train['month'] = df_train['visit_date'].dt.month
df_train['visit_date'] = df_train['visit_date'].dt.date

In [ ]:
df_train.head()

In [ ]:
df_test['visit_date'] = pd.to_datetime(df_test['visit_date'])
df_test['dow'] = df_test['visit_date'].dt.dayofweek
df_test['year'] = df_test['visit_date'].dt.year
df_test['month'] = df_test['visit_date'].dt.month
df_test['visit_date'] = df_test['visit_date'].dt.date

In [ ]:
df_test.head()

In [ ]:
unique_stores = df_test['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

stores.head()

In [ ]:
tmp = df_train.groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = df_train.groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = df_train.groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = df_train.groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = df_train.groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})

In [ ]:
stores.head()

In [ ]:
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
stores.head()

In [ ]:
stores = pd.merge(stores, air_store_info, how='left', on=['air_store_id']) 
stores.head()

In [ ]:
from sklearn import *
lbl = preprocessing.LabelEncoder()
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])
stores.head()

In [ ]:
date_info = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
date_info.head()

In [ ]:
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
date_info['day_of_week'] = lbl.fit_transform(date_info['day_of_week'])
date_info['visit_date'] = date_info['visit_date'].dt.date
date_info.head()

In [ ]:
train = pd.merge(df_train, date_info, how='left', on=['visit_date']) 
test = pd.merge(df_test, date_info, how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

train.head()

In [ ]:
test.head()

In [ ]:
air_reserve.head()

In [ ]:
store_id_relation.head()

In [ ]:
hpg_reserve = pd.merge(hpg_reserve, store_id_relation, how='inner', on=['hpg_store_id'])
hpg_reserve.head()

In [ ]:
train = pd.merge(train, hpg_reserve, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, hpg_reserve, how='left', on=['air_store_id','visit_date'])

train = pd.merge(train, air_reserve, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, air_reserve, how='left', on=['air_store_id','visit_date']) 

In [ ]:
train.head()

# TO BE CONTINUED !!